In [5]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate
pd.set_option("display.precision", 1)

In [6]:
import csv
import pandas as pd
import random

# Fungsi untuk generate data dummy
def generate_dummy_data():
    jumlah_favorit = random.randint(0, 1000)
    jumlah_like = random.randint(0, 10000)
    return {
        'jumlah_favorit': jumlah_favorit,
        'jumlah_like': jumlah_like
    }

# Generate 500 data dummy
dummy_data_list = [generate_dummy_data() for _ in range(50884)]

# Konversi ke DataFrame
dummy_data = pd.DataFrame(dummy_data_list)

In [7]:
dummy_data

,jumlah_favorit,jumlah_like
0,130,8339
1,103,9837
2,206,2570
3,64,2231
4,600,2384
...,...,...
50879,309,8703
50880,515,4928
50881,184,8843
50882,460,3194


In [8]:
# Specify the path to your CSV file
user_train_df = 'content_user_train.csv'

# Read the CSV file into a pandas DataFrame
df_user_train = pd.read_csv(user_train_df, header=None)

# Ganti kolom kedua atau isi matriks user_train menjadi jumlah_like dan jumlah_favorite
df_user_train.iloc[:, 1] = dummy_data['jumlah_favorit']
df_user_train.iloc[:, 2] = dummy_data['jumlah_like']

df_user_train = np.array(df_user_train)

<ipython-input-8-c23d4b1aef81>:8: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_user_train.iloc[:, 1] = dummy_data['jumlah_favorit']
<ipython-input-8-c23d4b1aef81>:9: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_user_train.iloc[:, 2] = dummy_data['jumlah_like']


In [9]:
import pandas as pd
from io import StringIO

# Assuming item_train is a NumPy array
user_train_csv = "\n".join(",".join(map(str, row)) for row in df_user_train)

# Membaca data string ke DataFrame
user_train_pd = pd.read_csv(StringIO(user_train_csv), header=None, names=["user_id", "jumlah_favorit", "jumlah_like", "Alam", "Petualangan", "Kuliner", "Ekstrem", "Seni", "Wisata", "Sejarah", "Festival", "Musik", "Urban", "Olahraga", "Kesehatan", "Lingkungan", "Pendidikan"])

# Menggunakan groupby dan transform untuk mengganti nilai kolom yang sama dengan yang pertama
user_train_pd[["jumlah_favorit", "jumlah_like"]] = user_train_pd.groupby("user_id")[["jumlah_favorit", "jumlah_like"]].transform("first")

user_train_pd = np.array(user_train_pd)

In [10]:
import pandas as pd
from io import StringIO

# Assuming user_train_pd is a NumPy array
df_user_train_csv = "\n".join(",".join(map(str, row)) for row in user_train_pd)

# Read the CSV data into a DataFrame
df_user_train_pd = pd.read_csv(StringIO(df_user_train_csv), header=None, names=["user_id", "jumlah_favorit", "jumlah_like", "Alam", "Petualangan", "Kuliner", "Ekstrem", "Seni", "Wisata", "Sejarah", "Festival", "Musik", "Urban", "Olahraga", "Kesehatan", "Lingkungan", "Pendidikan"])

# Ganti nilai selain 0.0 dengan 1
df_user_train_pd.iloc[:, 3:] = df_user_train_pd.iloc[:, 3:].applymap(lambda x: 1 if x != 0.0 else 0.0)

# Tampilkan DataFrame hasil
print(df_user_train_pd)

       user_id  jumlah_favorit  jumlah_like  Alam  Petualangan  Kuliner  \
0          2.0           130.0       8339.0   1.0          1.0      0.0   
1          2.0           130.0       8339.0   1.0          1.0      0.0   
2          2.0           130.0       8339.0   1.0          1.0      0.0   
3          2.0           130.0       8339.0   1.0          1.0      0.0   
4          2.0           130.0       8339.0   1.0          1.0      0.0   
...        ...             ...          ...   ...          ...      ...   
50879    610.0           902.0       6987.0   1.0          1.0      1.0   
50880    610.0           902.0       6987.0   1.0          1.0      1.0   
50881    610.0           902.0       6987.0   1.0          1.0      1.0   
50882    610.0           902.0       6987.0   1.0          1.0      1.0   
50883    610.0           902.0       6987.0   1.0          1.0      1.0   

       Ekstrem  Seni  Wisata  Sejarah  Festival  Musik  Urban  Olahraga  \
0          0.0   1.0    

In [11]:
# Set seed for reproducibility
np.random.seed(42)

# Iterate through rows
for index, row in df_user_train_pd.iterrows():
    # Find indices where the value is 1
    ones_indices = np.where(row.values[3:] == 1)[0]

    # Check if there are any ones
    if len(ones_indices) > 0:
        # Generate random values for the ones
        random_values = np.random.randint(1, 10, len(ones_indices))

        # Calculate scaling factor based on "jumlah_like"
        total_like = row["jumlah_like"]
        total_random = np.sum(random_values)
        scaling_factor = total_like / total_random

        # Scale random values
        random_values_scaled = (random_values * scaling_factor).astype(int)

        # Set the values in the DataFrame
        df_user_train_pd.iloc[index, 3:] = 0
        df_user_train_pd.iloc[index, 3 + ones_indices] = random_values_scaled

# Display the DataFrame
print(df_user_train_pd)

       user_id  jumlah_favorit  jumlah_like    Alam  Petualangan  Kuliner  \
0          2.0           130.0       8339.0  1006.0        575.0      0.0   
1          2.0           130.0       8339.0  1258.0       1258.0      0.0   
2          2.0           130.0       8339.0   173.0       1042.0      0.0   
3          2.0           130.0       8339.0   438.0       1024.0      0.0   
4          2.0           130.0       8339.0   947.0        379.0      0.0   
...        ...             ...          ...     ...          ...      ...   
50879    610.0           902.0       6987.0   652.0        652.0    652.0   
50880    610.0           902.0       6987.0   249.0        873.0    499.0   
50881    610.0           902.0       6987.0   349.0        232.0    931.0   
50882    610.0           902.0       6987.0   566.0        566.0    472.0   
50883    610.0           902.0       6987.0   607.0        101.0    405.0   

       Ekstrem    Seni  Wisata  Sejarah  Festival  Musik   Urban  Olahraga 

In [12]:
# Replace "jumlah_like" with the sum of values from "Alam" through "Pendidikan"
df_user_train_pd["jumlah_like"] = df_user_train_pd.iloc[:, 3:].sum(axis=1)

# Display the updated DataFrame
print(df_user_train_pd)

       user_id  jumlah_favorit  jumlah_like    Alam  Petualangan  Kuliner  \
0          2.0           130.0       8335.0  1006.0        575.0      0.0   
1          2.0           130.0       8334.0  1258.0       1258.0      0.0   
2          2.0           130.0       8334.0   173.0       1042.0      0.0   
3          2.0           130.0       8334.0   438.0       1024.0      0.0   
4          2.0           130.0       8336.0   947.0        379.0      0.0   
...        ...             ...          ...     ...          ...      ...   
50879    610.0           902.0       6981.0   652.0        652.0    652.0   
50880    610.0           902.0       6980.0   249.0        873.0    499.0   
50881    610.0           902.0       6980.0   349.0        232.0    931.0   
50882    610.0           902.0       6980.0   566.0        566.0    472.0   
50883    610.0           902.0       6981.0   607.0        101.0    405.0   

       Ekstrem    Seni  Wisata  Sejarah  Festival  Musik   Urban  Olahraga 

In [13]:
# Group by 'user_id' and fill 'jumlah_like' to 'Pendidikan' columns with the first encountered value
df_user_train_pd[['jumlah_like', 'Alam', 'Petualangan', 'Kuliner', 'Ekstrem', 'Seni', 'Wisata', 'Sejarah',
                   'Festival', 'Musik', 'Urban', 'Olahraga', 'Kesehatan', 'Lingkungan', 'Pendidikan']] = \
    df_user_train_pd.groupby('user_id', as_index=False)[['jumlah_like', 'Alam', 'Petualangan', 'Kuliner',
                                                         'Ekstrem', 'Seni', 'Wisata', 'Sejarah', 'Festival',
                                                         'Musik', 'Urban', 'Olahraga', 'Kesehatan', 'Lingkungan',
                                                         'Pendidikan']].transform('first')

user_train = np.array(df_user_train_pd)

In [14]:
user_train

array([[2.000e+00, 1.300e+02, 8.335e+03, ..., 0.000e+00, 7.180e+02,
        5.750e+02],
       [2.000e+00, 1.300e+02, 8.335e+03, ..., 0.000e+00, 7.180e+02,
        5.750e+02],
       [2.000e+00, 1.300e+02, 8.335e+03, ..., 0.000e+00, 7.180e+02,
        5.750e+02],
       ...,
       [6.100e+02, 9.020e+02, 6.987e+03, ..., 5.480e+02, 2.740e+02,
        2.740e+02],
       [6.100e+02, 9.020e+02, 6.987e+03, ..., 5.480e+02, 2.740e+02,
        2.740e+02],
       [6.100e+02, 9.020e+02, 6.987e+03, ..., 5.480e+02, 2.740e+02,
        2.740e+02]])

In [15]:
user_train.shape

(50884, 17)

In [16]:
# Specify the path to your CSV file
item_train_df = 'content_item_train.csv'

# Read the CSV file into a pandas DataFrame
df_item_train = pd.read_csv(item_train_df, header=None)

# Ganti kolom kedua atau isi matriks user_train menjadi jumlah_like dan jumlah_favorite
df_item_train.iloc[:, 1] = dummy_data['jumlah_favorit']
df_item_train.iloc[:, 2] = dummy_data['jumlah_like']

df_item_train = np.array(df_item_train)

<ipython-input-16-b99d56f0909f>:9: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_item_train.iloc[:, 2] = dummy_data['jumlah_like']


In [17]:
df_item_train

array([[  6874,    130,   8339, ...,      0,      0,      1],
       [  8798,    103,   9837, ...,      0,      0,      1],
       [ 46970,    206,   2570, ...,      0,      0,      0],
       ...,
       [168250,    184,   8843, ...,      0,      0,      0],
       [168250,    460,   3194, ...,      0,      0,      0],
       [168252,    534,   2556, ...,      0,      1,      0]])

In [18]:
import pandas as pd
from io import StringIO

# Assuming item_train is a NumPy array
item_train_csv = "\n".join(",".join(map(str, row)) for row in df_item_train)

# Membaca data string ke DataFrame
item_train = pd.read_csv(StringIO(item_train_csv), header=None, names=["item_id", "jumlah_favorit", "jumlah_like", "Alam", "Petualangan", "Kuliner", "Ekstrem", "Seni", "Wisata", "Sejarah", "Festival", "Musik", "Urban", "Olahraga", "Kesehatan", "Lingkungan", "Pendidikan"])

# Menggunakan groupby dan transform untuk mengganti nilai kolom yang sama dengan yang pertama
item_train[["jumlah_favorit", "jumlah_like"]] = item_train.groupby("item_id")[["jumlah_favorit", "jumlah_like"]].transform("first")

item_train = np.array(item_train)

In [19]:
item_train

array([[  6874,    130,   8339, ...,      0,      0,      1],
       [  8798,    103,   9837, ...,      0,      0,      1],
       [ 46970,    206,   2570, ...,      0,      0,      0],
       ...,
       [168250,    100,   3260, ...,      0,      0,      0],
       [168250,    100,   3260, ...,      0,      0,      0],
       [168252,    917,    817, ...,      0,      1,      0]])

In [20]:
item_train.shape

(50884, 17)

In [21]:
from io import StringIO

# Assuming item_train[:, 2] contains the CSV data
y_train_csv = "\n".join(map(str, item_train[:, 2]))

# Read the CSV data into a pandas DataFrame
y_train = pd.read_csv(StringIO(y_train_csv), header=None)
y_train = np.array(y_train).reshape(-1)

In [22]:
y_train

array([8339, 9837, 2570, ..., 3260, 3260,  817])

In [23]:
y_train.shape

(50884,)

In [24]:
item_features = ["item_id", "jumlah_favorit", "jumlah_like", "Alam", "Petualangan", "Kuliner", "Ekstrem", "Seni", "Wisata", "Sejarah", "Festival", "Musik", "Urban", "Olahraga", "Kesehatan", "Lingkungan", "Pendidikan"]

In [25]:
item_features

['item_id',
 'jumlah_favorit',
 'jumlah_like',
 'Alam',
 'Petualangan',
 'Kuliner',
 'Ekstrem',
 'Seni',
 'Wisata',
 'Sejarah',
 'Festival',
 'Musik',
 'Urban',
 'Olahraga',
 'Kesehatan',
 'Lingkungan',
 'Pendidikan']

In [26]:
user_features = ["user_id", "jumlah_favorit", "jumlah_like", "Alam", "Petualangan", "Kuliner", "Ekstrem", "Seni", "Wisata", "Sejarah", "Festival", "Musik", "Urban", "Olahraga", "Kesehatan", "Lingkungan", "Pendidikan"]

In [27]:
user_features

['user_id',
 'jumlah_favorit',
 'jumlah_like',
 'Alam',
 'Petualangan',
 'Kuliner',
 'Ekstrem',
 'Seni',
 'Wisata',
 'Sejarah',
 'Festival',
 'Musik',
 'Urban',
 'Olahraga',
 'Kesehatan',
 'Lingkungan',
 'Pendidikan']

In [28]:
import pandas as pd
import numpy as np
from io import StringIO

# Assuming item_train is a NumPy array
item_vecs_csv = "\n".join(",".join(map(str, row)) for row in item_train)

# Membaca data string ke DataFrame
item_vecs = pd.read_csv(StringIO(item_vecs_csv), header=None, names=["item_id", "jumlah_favorit", "jumlah_like", "Alam", "Petualangan", "Kuliner", "Ekstrem", "Seni", "Wisata", "Sejarah", "Festival", "Musik", "Urban", "Olahraga", "Kesehatan", "Lingkungan", "Pendidikan"])

# Drop duplicates based on "item_id" and keep the first occurrence
item_vecs = item_vecs.drop_duplicates(subset="item_id", keep="first")

# Sort the array by "item_id"
item_vecs = item_vecs.sort_values(by="item_id")

# Convert the DataFrame to a NumPy array
item_vecs = np.array(item_vecs)

In [29]:
item_vecs

array([[  4054,    800,    673, ...,      1,      0,      0],
       [  4069,    668,    969, ...,      1,      0,      0],
       [  4148,    842,   4693, ...,      0,      0,      1],
       ...,
       [177765,    716,   6475, ...,      0,      0,      0],
       [179819,    752,   3806, ...,      0,      1,      0],
       [187593,    108,   4027, ...,      0,      1,      0]])

In [30]:
item_vecs.shape

(847, 17)

In [31]:
num_user_features = user_train.shape[1] - 3  # remove userid, rating count and ave rating during training
num_item_features = item_train.shape[1] - 1  # remove movie id at train time
uvs = 3  # user genre vector start
ivs = 3  # item genre vector start
u_s = 3  # start of columns to use in training, user
i_s = 1  # start of columns to use in training, items
print(f"Number of training vectors: {len(item_train)}")

Number of training vectors: 50884


In [32]:
print(f"y_train[:5]: {y_train[:5]}")

y_train[:5]: [8339 9837 2570 2231 2384]


In [33]:
# scale training data
item_train_unscaled = item_train
user_train_unscaled = user_train
y_train_unscaled    = y_train

scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.reshape(-1, 1))
#ynorm_test = scalerTarget.transform(y_test.reshape(-1, 1))

print(np.allclose(item_train_unscaled, scalerItem.inverse_transform(item_train)))
print(np.allclose(user_train_unscaled, scalerUser.inverse_transform(user_train)))

True
True


In [34]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"item training data shape: {item_train.shape}")
print(f"item test data shape: {item_test.shape}")

item training data shape: (40707, 17)
item test data shape: (10177, 17)


In [35]:
num_outputs = 32
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([
    # Input layer
    tf.keras.layers.InputLayer(input_shape=(num_user_features)),
    # Fully connected layer with 256 units and ReLU activation
    tf.keras.layers.Dense(256, activation='relu'),
    # Fully connected layer with 128 units and ReLU activation
    tf.keras.layers.Dense(128, activation='relu'),
    # Fully connected layer with num_outputs units and no activation
    tf.keras.layers.Dense(num_outputs)
])

item_NN = tf.keras.models.Sequential([
    # Input layer
    tf.keras.layers.InputLayer(input_shape=(num_item_features)),
    # Fully connected layer with 256 units and ReLU activation
    tf.keras.layers.Dense(256, activation='relu'),
    # Fully connected layer with 128 units and ReLU activation
    tf.keras.layers.Dense(128, activation='relu'),
    # Fully connected layer with num_outputs units and no activation
    tf.keras.layers.Dense(num_outputs)
])

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model
model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 14)]                 0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 16)]                 0         []                            
                                                                                                  
 sequential (Sequential)     (None, 32)                   40864     ['input_3[0][0]']             
                                                                                                  
 sequential_1 (Sequential)   (None, 32)                   41376     ['input_4[0][0]']             
                                                                                              

In [36]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

In [37]:
tf.random.set_seed(1)
model.fit([user_train[:, u_s:], item_train[:, i_s:]], y_train, epochs=30)

Epoch 1/30
1273/1273 [==============================] - 7s 4ms/step - loss: 0.0038
Epoch 2/30
1273/1273 [==============================] - 4s 3ms/step - loss: 4.4100e-04
Epoch 3/30
1273/1273 [==============================] - 4s 3ms/step - loss: 4.5577e-04
Epoch 4/30
1273/1273 [==============================] - 5s 4ms/step - loss: 2.5271e-04
Epoch 5/30
1273/1273 [==============================] - 4s 3ms/step - loss: 2.4871e-04
Epoch 6/30
1273/1273 [==============================] - 3s 3ms/step - loss: 1.9989e-04
Epoch 7/30
1273/1273 [==============================] - 4s 3ms/step - loss: 1.8750e-04
Epoch 8/30
1273/1273 [==============================] - 6s 5ms/step - loss: 1.5760e-04
Epoch 9/30
1273/1273 [==============================] - 5s 4ms/step - loss: 1.2945e-04
Epoch 10/30
1273/1273 [==============================] - 4s 3ms/step - loss: 1.1503e-04
Epoch 11/30
1273/1273 [==============================] - 5s 4ms/step - loss: 1.0035e-04
Epoch 12/30
1273/1273 [======================

In [38]:
model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], y_test)

319/319 [==============================] - 1s 2ms/step - loss: 3.8564e-05


3.856419789372012e-05

In [39]:
new_user_id = 5000
new_jumlah_like = 1000
new_alam = 100
new_petualangan = 200
new_kuliner = 0.0
new_ekstrem = 300
new_seni = 0.0
new_wisata = 0.0
new_sejarah = 0.0
new_festival = 0.0
new_musik = 300
new_urban = 0.0
new_olahraga = 100
new_kesehatan = 0.0
new_lingkungan = 0.0
new_pendidikan = 0.0
new_jumlah_favorit = 100

user_vec = np.array([[new_user_id, new_jumlah_favorit, new_jumlah_like,
                      new_alam, new_petualangan, new_kuliner, new_ekstrem,
                      new_seni, new_wisata, new_sejarah, new_festival,
                      new_musik, new_urban, new_olahraga, new_kesehatan,
                      new_lingkungan, new_pendidikan]])

In [40]:
!pip install pickle5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=255321 sha256=4a245ca3538c4f6bfb546cd954897bf7072367bdac43c12ff196f3e721b7c726
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5


In [41]:
!pip install tabulate

In [42]:
from collections import defaultdict
import csv
import numpy as np
from numpy import genfromtxt
import pickle5 as pickle
import tabulate
from IPython.display import HTML

In [43]:
def load_data():
    post_dict = defaultdict(dict)
    count = 0
#    with open('./data/movies.csv', newline='') as csvfile:
    with open('post_list.csv', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for line in reader:
            if count == 0:
                count += 1  #skip header
                #print(line) print
            else:
                count += 1
                item_id = int(line[0])
                post_dict[item_id]["judul_postingan"] = line[1]
                post_dict[item_id]["type"] = line[2]

    return post_dict

post_dict = load_data()

In [44]:
def gen_user_vecs(user_vec, num_items):
    """ given a user vector return:
        user predict maxtrix to match the size of item_vecs """
    user_vecs = np.tile(user_vec, (num_items, 1))
    return user_vecs

def print_pred_movies(y_p, item, post_dict, maxcount=10):
    """ print results of prediction of a new user. inputs are expected to be in
        sorted order, unscaled. """
    count = 0
    disp = [["y_p", "item_id", "jumlah_like", "judul_postingan", "type"]]

    for i in range(0, y_p.shape[0]):
        if count == maxcount:
            break
        count += 1
        item_id = item[i, 0].astype(int)
        disp.append([np.around(y_p[i, 0], 1), item[i, 0].astype(int), np.around(item[i, 2].astype(float), 1),
                     post_dict[item_id]['judul_postingan'], post_dict[item_id]['type']])

    table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow")
    return table

# generate and replicate the user vector to match the number movies in the data set.
user_vecs = gen_user_vecs(user_vec,len(item_vecs))

# scale our user and item vectors
suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)

# make a prediction
y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

# unscale y prediction
y_pu = scalerTarget.inverse_transform(y_p)

# sort the results, highest prediction first
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]  #using unscaled vectors for display

HTML(print_pred_movies(sorted_ypu, sorted_items, post_dict, maxcount=10))

27/27 [==============================] - 0s 3ms/step


y_p,item_id,jumlah_like,judul_postingan,type
9948.7,55768,9961,Nikmati di curug tempat hiburan taman pintar,sejarah
9927.5,6711,9940,Menyaksikan di hutan kebun binatang taman pintar,alam| seni
9927.4,54190,9980,Nyantai di di pantai taman bermain taman pintar,petualangan| kuliner| extreme| seni| music| kesehatan
9908.5,4306,9903,Temukan Hal Unik di di pantai tempat hiburan perpustakaan,petualangan| kuliner| seni| music| kesehatan
9906.5,6218,9885,Amati di pantai tempat hiburan taman pintar,seni| sejarah| festival
9888.5,54259,9871,Menyaksikan di pantai taman bermain taman pintar,festival| olahraga| lingkungan| pendidikan
9880,8370,9895,Rasakan Sensasi Tak Terlupakan di di pantai taman bermain museum pendidikan,festival
9872.9,27904,9885,Saksikan Keindahan di hutan waterpark museum pendidikan,petualangan| kuliner| extreme| seni| music
9849.6,88163,9830,Petualangan di di pantai tempat hiburan perpustakaan,petualangan| kuliner| extreme| seni
9840.5,8783,9830,Liburan ke di hutan waterpark perpustakaan,wisata| festival| pendidikan


In [45]:
def sq_dist(a, b):
    """
    Returns the squared distance between two vectors
    Args:
      a (ndarray (n,)): vector with n features
      b (ndarray (n,)): vector with n features
    Returns:
      d (float): squared distance
    """
    ### START CODE HERE ###
    d = np.sum(np.square(a - b))
    ### END CODE HERE ###
    return d

In [46]:
a1 = np.array([1.0, 2.0, 3.0]); b1 = np.array([1.0, 2.0, 3.0])
a2 = np.array([1.1, 2.1, 3.1]); b2 = np.array([1.0, 2.0, 3.0])
a3 = np.array([0, 1, 0]);       b3 = np.array([1, 0, 0])
print(f"squared distance between a1 and b1: {sq_dist(a1, b1):0.3f}")
print(f"squared distance between a2 and b2: {sq_dist(a2, b2):0.3f}")
print(f"squared distance between a3 and b3: {sq_dist(a3, b3):0.3f}")

squared distance between a1 and b1: 0.000
squared distance between a2 and b2: 0.030
squared distance between a3 and b3: 2.000


In [47]:
input_item_m = tf.keras.layers.Input(shape=(num_item_features))    # input layer
vm_m = item_NN(input_item_m)                                       # use the trained item_NN
vm_m = tf.linalg.l2_normalize(vm_m, axis=1)                        # incorporate normalization as was done in the original model
model_m = tf.keras.Model(input_item_m, vm_m)
model_m.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 16)]              0         
                                                                 
 sequential_1 (Sequential)   (None, 32)                41376     
                                                                 
 tf.math.l2_normalize_2 (TF  (None, 32)                0         
 OpLambda)                                                       
                                                                 
Total params: 41376 (161.62 KB)
Trainable params: 41376 (161.62 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [48]:
scaled_item_vecs = scalerItem.transform(item_vecs)
vms = model_m.predict(scaled_item_vecs[:,i_s:])
print(f"size of all predicted post feature vectors: {vms.shape}")

27/27 [==============================] - 0s 2ms/step
size of all predicted post feature vectors: (847, 32)


In [49]:
count = 50  # number of movies to display
dim = len(vms)
dist = np.zeros((dim,dim))

for i in range(dim):
    for j in range(dim):
        dist[i,j] = sq_dist(vms[i, :], vms[j, :])

m_dist = ma.masked_array(dist, mask=np.identity(dist.shape[0]))  # mask the diagonal

disp = [["post1", "type", "post2", "type"]]
for i in range(count):
    min_idx = np.argmin(m_dist[i])
    post1_id = int(item_vecs[i,0])
    post2_id = int(item_vecs[min_idx,0])
    disp.append( [post_dict[post1_id]['judul_postingan'], post_dict[post1_id]['type'],
                  post_dict[post2_id]['judul_postingan'], post_dict[post1_id]['type']]
               )
table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow")
HTML(table)

post1,type,post2,type
Eksklusivitas di pegunungan tempat hiburan taman pintar,seni| sejarah,Pengalaman Seru di di danau kebun binatang taman pintar,seni| sejarah
Lihat Keindahan di hutan tempat hiburan museum pendidikan,alam| wisata| festival| olahraga| lingkungan| pendidikan,Jelajahi di pegunungan kebun binatang taman pintar,alam| wisata| festival| olahraga| lingkungan| pendidikan
Healing ke di pegunungan taman bermain perpustakaan,petualangan| kuliner| extreme| seni| music| lingkungan,Jalan-jalan ke di curug tempat hiburan taman pintar,petualangan| kuliner| extreme| seni| music| lingkungan
Lihat Keindahan di pegunungan taman bermain taman pintar,wisata| festival,Temukan Hal Unik di di pantai waterpark perpustakaan,wisata| festival
Traveling ke di danau waterpark museum pendidikan,seni| kesehatan,Jangan Lewatkan Pengalaman Seru di di pantai tempat hiburan taman pintar,seni| kesehatan
Healing ke di pegunungan kebun binatang museum pendidikan,petualangan| music,Jajal di curug tempat hiburan perpustakaan,petualangan| music
Lewati Perjalanan Menakjubkan ke di hutan kebun binatang perpustakaan,petualangan| kuliner| extreme| seni,Jalan-jalan ke di hutan waterpark taman pintar,petualangan| kuliner| extreme| seni
Jangan Hanya Lihat di pegunungan taman bermain taman pintar,seni| festival| kesehatan,Pesona di pantai waterpark perpustakaan,seni| festival| kesehatan
Jalan-jalan ke di hutan waterpark museum pendidikan,petualangan| music| olahraga| kesehatan,Nyantai di di pantai taman bermain perpustakaan,petualangan| music| olahraga| kesehatan
Eksklusivitas di danau kebun binatang perpustakaan,alam| petualangan| kuliner| extreme| seni,Hampiri di pantai taman bermain museum pendidikan,alam| petualangan| kuliner| extreme| seni


In [50]:
model.save("/content/drive/MyDrive/Post and User Preferences Recommendation/Post and User Preferences Recommendation.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
